<a href="https://colab.research.google.com/github/sevenjunebaby/AiModels/blob/main/Compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing randomized search and grid search for hyperparameter estimation

In [1]:
from time import time

import numpy as np
import scipy.stats as stats

from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# get some data
X, y = load_digits(return_X_y=True, n_class=3)

# build a classifier
clf = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": stats.loguniform(1e-2, 1e0),
}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

start = time()
random_search.fit(X, y)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)
report(grid_search.cv_results_)

RandomizedSearchCV took 6.41 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.993 (std: 0.007)
Parameters: {'alpha': np.float64(0.05690276955830174), 'average': False, 'l1_ratio': np.float64(0.13822256090399687)}

Model with rank: 2
Mean validation score: 0.983 (std: 0.017)
Parameters: {'alpha': np.float64(0.09460058024709554), 'average': False, 'l1_ratio': np.float64(0.7468127201182538)}

Model with rank: 2
Mean validation score: 0.983 (std: 0.012)
Parameters: {'alpha': np.float64(0.02191695691943531), 'average': False, 'l1_ratio': np.float64(0.9912520987971161)}

GridSearchCV took 13.24 seconds for 60 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.994 (std: 0.007)
Parameters: {'alpha': np.float64(0.01), 'average': False, 'l1_ratio': np.float64(0.8888888888888888)}

Model with rank: 2
Mean validation score: 0.993 (std: 0.011)
Parameters: {'alpha': np.float64(0.01), 'average': False, 'l1_ratio': np.float64(1.0)}

Model